In [ ]:
import asyncio
import json
from typing import Dict, List, Optional, Set, Union
from enum import Enum
from datetime import datetime
from pydantic import BaseModel, Field
from collections import deque

# ----- Data Models -----

class ExpertiseArea(str, Enum):
    FRONTEND = "frontend"
    BACKEND = "backend"
    DATABASE = "database"
    DEVOPS = "devops"
    TESTING = "testing"
    SECURITY = "security"

class TaskStatus(str, Enum):
    PENDING = "pending"
    ASSIGNED = "assigned"
    IN_PROGRESS = "in_progress"
    REVIEW = "review"
    COMPLETED = "completed"
    BLOCKED = "blocked"

class MessageType(str, Enum):
    TASK_ASSIGNMENT = "task_assignment"
    STATUS_UPDATE = "status_update"
    TECHNICAL_QUERY = "technical_query"
    FEEDBACK = "feedback"
    LEARNING = "learning"
    PROGRESS_REPORT = "progress_report"

class Task(BaseModel):
    id: str
    title: str
    description: str
    required_expertise: List[ExpertiseArea]
    status: TaskStatus = TaskStatus.PENDING
    assigned_to: Optional[str] = None
    priority: int = 1
    estimated_hours: float = 0
    deadline: Optional[datetime] = None
    dependencies: List[str] = Field(default_factory=list)
    subtasks: List[str] = Field(default_factory=list)

class Message(BaseModel):
    from_agent: str
    to_agent: str
    message_type: MessageType
    content: Dict
    timestamp: datetime = Field(default_factory=datetime.now)

class ProgrammerMetrics(BaseModel):
    tasks_completed: int = 0
    average_task_time: float = 0
    skill_improvements: Dict[str, float] = Field(default_factory=dict)
    learning_activities: List[str] = Field(default_factory=list)

class LearningContext(BaseModel):
    expertise_area: ExpertiseArea
    learning_objectives: List[str]
    required_skills: List[str]
    resources: List[str]
    progress: float = 0.0

# ----- Project Manager Agent -----

class ProjectManagerAgent:
    def __init__(self, agent_id: str, model_name: str = "phi-3-mini"):
        self.agent_id = agent_id
        self.model_name = model_name
        self.tasks = {}  # task_id -> Task
        self.team = {}   # agent_id -> ProgrammerAgent info
        self.message_queue = asyncio.Queue()
        self.sent_messages = []
        self.received_messages = []
        
        # Task management
        self.pending_tasks = []
        self.in_progress_tasks = []
        self.completed_tasks = []
        
        # Learning management
        self.learning_contexts = {}  # agent_id -> list of LearningContext
        self.team_metrics = {}  # agent_id -> ProgrammerMetrics
        
        # Knowledge base references
        self.manual_references = {}  # topic -> document reference
        
        # Internal thought log
        self.thought_log = deque(maxlen=100)
    
    async def start(self):
        """Start the agent's main loop"""
        await asyncio.gather(
            self.process_messages(),
            self.manage_tasks(),
            self.monitor_team_progress()
        )
    
    async def process_messages(self):
        """Process incoming messages from the queue"""
        while True:
            message = await self.message_queue.get()
            self.received_messages.append(message)
            
            # Log internal thought about message
            self.add_thought(f"Received {message.message_type} from {message.from_agent}")
            
            if message.message_type == MessageType.STATUS_UPDATE:
                await self.handle_status_update(message)
            elif message.message_type == MessageType.TECHNICAL_QUERY:
                await self.forward_to_architect(message)
            elif message.message_type == MessageType.PROGRESS_REPORT:
                await self.update_team_metrics(message)
            
            self.message_queue.task_done()
    
    async def manage_tasks(self):
        """Periodically check and manage tasks"""
        while True:
            # Assign any pending tasks
            for task_id in list(self.pending_tasks):
                task = self.tasks[task_id]
                if all(dep_id not in self.pending_tasks and dep_id not in self.in_progress_tasks 
                       for dep_id in task.dependencies):
                    await self.assign_task(task)
                    self.pending_tasks.remove(task_id)
            
            # Check for overdue tasks
            current_time = datetime.now()
            for task_id in self.in_progress_tasks:
                task = self.tasks[task_id]
                if task.deadline and current_time > task.deadline:
                    self.add_thought(f"Task {task.id} is overdue, needs attention")
                    await self.send_reminder(task)
            
            await asyncio.sleep(60)  # Check every minute
    
    async def monitor_team_progress(self):
        """Monitor team progress and suggest learning opportunities"""
        while True:
            for agent_id, metrics in self.team_metrics.items():
                if metrics.tasks_completed > 0 and metrics.tasks_completed % 5 == 0:
                    # Suggest learning after every 5 completed tasks
                    await self.suggest_learning(agent_id)
            
            # Generate team progress summary
            self.add_thought("Generated team progress summary")
            
            await asyncio.sleep(3600)  # Check every hour
    
    async def assign_task(self, task: Task):
        """Assign a task to the most suitable programmer"""
        best_programmer = None
        best_score = -1
        
        for agent_id, info in self.team.items():
            # Skip if programmer is at capacity
            if len(info.get("current_tasks", [])) >= info.get("capacity", 3):
                continue
            
            # Calculate suitability score
            expertise_match = sum(1 for exp in task.required_expertise if exp in info.get("expertise", []))
            workload = len(info.get("current_tasks", []))
            experience = self.team_metrics.get(agent_id, ProgrammerMetrics()).tasks_completed
            
            # Simple scoring formula
            score = expertise_match * 3 - workload + (experience * 0.1)
            
            if score > best_score:
                best_score = score
                best_programmer = agent_id
        
        if best_programmer:
            # Update task status
            task.status = TaskStatus.ASSIGNED
            task.assigned_to = best_programmer
            self.in_progress_tasks.append(task.id)
            
            # Notify programmer
            await self.send_message(
                best_programmer,
                MessageType.TASK_ASSIGNMENT,
                {"task": task.dict()}
            )
            
            self.add_thought(f"Assigned task {task.id} to {best_programmer} with score {best_score}")
        else:
            self.add_thought(f"Could not find suitable programmer for task {task.id}")
            # Keep in pending for now
            task.status = TaskStatus.PENDING
    
    async def handle_status_update(self, message: Message):
        """Handle status updates from programmers"""
        content = message.content
        task_id = content.get("task_id")
        new_status = content.get("status")
        
        if task_id in self.tasks:
            task = self.tasks[task_id]
            old_status = task.status
            task.status = TaskStatus(new_status)
            
            self.add_thought(f"Task {task_id} status changed from {old_status} to {new_status}")
            
            # If completed, move to completed list
            if task.status == TaskStatus.COMPLETED:
                self.in_progress_tasks.remove(task_id)
                self.completed_tasks.append(task_id)
                
                # Update metrics
                agent_id = task.assigned_to
                if agent_id in self.team_metrics:
                    metrics = self.team_metrics[agent_id]
                    metrics.tasks_completed += 1
                    # Could update average_task_time here if we tracked start time
                
                # Provide feedback
                await self.send_feedback(task)
    
    async def send_message(self, to_agent: str, message_type: MessageType, content: Dict):
        """Send a message to another agent"""
        message = Message(
            from_agent=self.agent_id,
            to_agent=to_agent,
            message_type=message_type,
            content=content
        )
        
        self.sent_messages.append(message)
        # In a real implementation, this would use a message broker or direct API call
        print(f"[{self.agent_id}] Sending {message_type} to {to_agent}")
        
        # Simulate sending the message
        # In a real implementation, this would involve an API call or message queue
    
    async def forward_to_architect(self, message: Message):
        """Forward technical queries to the architect"""
        await self.send_message(
            "architect",
            MessageType.TECHNICAL_QUERY,
            {
                "original_from": message.from_agent,
                "query": message.content.get("query", ""),
                "context": message.content.get("context", {})
            }
        )
    
    async def send_reminder(self, task: Task):
        """Send a reminder about an overdue task"""
        if task.assigned_to:
            await self.send_message(
                task.assigned_to,
                MessageType.FEEDBACK,
                {
                    "task_id": task.id,
                    "reminder": f"Task '{task.title}' is past its deadline. Please provide a status update."
                }
            )
    
    async def send_feedback(self, task: Task):
        """Send feedback on a completed task"""
        if task.assigned_to:
            # In a real implementation, we would use the LLM to generate personalized feedback
            feedback = f"Good job completing '{task.title}'! Your implementation meets the requirements."
            
            await self.send_message(
                task.assigned_to,
                MessageType.FEEDBACK,
                {
                    "task_id": task.id,
                    "feedback": feedback,
                    "areas_for_improvement": []  # Would be generated by LLM
                }
            )
    
    async def suggest_learning(self, agent_id: str):
        """Suggest learning opportunities to a programmer"""
        if agent_id in self.team:
            # Get programmer's current expertise
            expertise = self.team[agent_id].get("expertise", [])
            
            # In a real implementation, we would use the LLM to generate personalized suggestions
            learning_context = LearningContext(
                expertise_area=expertise[0] if expertise else ExpertiseArea.TESTING,
                learning_objectives=["Improve command execution efficiency", "Learn error pattern recognition"],
                required_skills=["Pattern analysis", "System monitoring"],
                resources=["Command optimization guide", "Error pattern handbook"]
            )
            
            if agent_id not in self.learning_contexts:
                self.learning_contexts[agent_id] = []
            
            self.learning_contexts[agent_id].append(learning_context)
            
            await self.send_message(
                agent_id,
                MessageType.LEARNING,
                {"learning_context": learning_context.dict()}
            )
    
    async def update_team_metrics(self, message: Message):
        """Update team metrics based on progress reports"""
        agent_id = message.from_agent
        metrics = message.content.get("metrics", {})
        
        if agent_id not in self.team_metrics:
            self.team_metrics[agent_id] = ProgrammerMetrics()
        
        # Update metrics
        if "tasks_completed" in metrics:
            self.team_metrics[agent_id].tasks_completed = metrics["tasks_completed"]
        
        if "skill_improvements" in metrics:
            for skill, value in metrics["skill_improvements"].items():
                self.team_metrics[agent_id].skill_improvements[skill] = value
        
        if "learning_activities" in metrics:
            self.team_metrics[agent_id].learning_activities.extend(metrics["learning_activities"])
    
    def add_thought(self, thought: str):
        """Add an internal thought to the log"""
        self.thought_log.append({
            "timestamp": datetime.now().isoformat(),
            "thought": thought
        })
    
    async def register_programmer(self, agent_id: str, expertise: List[ExpertiseArea], capacity: int = 3):
        """Register a new programmer agent"""
        self.team[agent_id] = {
            "expertise": expertise,
            "capacity": capacity,
            "current_tasks": []
        }
        self.team_metrics[agent_id] = ProgrammerMetrics()
        
        self.add_thought(f"Registered programmer {agent_id} with expertise in {expertise}")
    
    async def add_task(self, task: Task):
        """Add a new task to the system"""
        self.tasks[task.id] = task
        self.pending_tasks.append(task.id)
        
        self.add_thought(f"Added new task {task.id}: {task.title}")
    
    async def add_manual_reference(self, topic: str, document_path: str):
        """Add reference to a manual document"""
        self.manual_references[topic] = document_path
    
    async def generate_report(self):
        """Generate a report on team progress"""
        # In a real implementation, we would use the LLM to generate a report
        report = {
            "timestamp": datetime.now().isoformat(),
            "completed_tasks": len(self.completed_tasks),
            "in_progress_tasks": len(self.in_progress_tasks),
            "pending_tasks": len(self.pending_tasks),
            "team_performance": {
                agent_id: {
                    "tasks_completed": metrics.tasks_completed,
                    "skill_improvements": metrics.skill_improvements
                }
                for agent_id, metrics in self.team_metrics.items()
            }
        }
        
        return report

# ----- Usage Example -----

async def example_usage():
    # Initialize the project manager
    manager = ProjectManagerAgent("project_manager_1")
    
    # Register programmers
    await manager.register_programmer(
        "programmer_1", 
        [ExpertiseArea.FRONTEND, ExpertiseArea.TESTING]
    )
    
    await manager.register_programmer(
        "programmer_2", 
        [ExpertiseArea.BACKEND, ExpertiseArea.DATABASE]
    )
    
    # Add tasks
    for i in range(5):
        task = Task(
            id=f"task_{i+1}",
            title=f"Implement feature {i+1}",
            description=f"Detailed description for feature {i+1}",
            required_expertise=[
                ExpertiseArea.FRONTEND if i % 2 == 0 else ExpertiseArea.BACKEND
            ]
        )
        await manager.add_task(task)
    
    # Start the manager
    await manager.start()

# Entry point for running the example
if __name__ == "__main__":
    asyncio.run(example_usage())